In [1]:
import folium
import os
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shapefile
import pickle
import random
import gams_magic

In [2]:
# read link data
road_file_path = os.path.join('..','road_file.csv')
road_file = pd.read_csv(road_file_path,names=["RDWY_LINK_ID","REF_SITE_FROM_ID","REF_SITE_TO_ID"])

# read road shape (shape file already converted using CRS in qgis)
shape_path = os.path.join("..","transportation_data","Middleton_Cross_Plains","Features","Middleton_Road_New.shp")
shape = shapefile.Reader(shape_path)

# read crash shape
crash_path = os.path.join("..","transportation_data","Middleton_Cross_Plains","Features","Crash_data_combined_2017_2020.shp")
crash = shapefile.Reader(crash_path)

# load reference point coordinate
reference_coordinate = None
reference_path = os.path.join("..","reference_coordinate")

with open(reference_path, 'rb') as f:
    reference_coordinate = pickle.load(f)

# Start Model

In [1]:
%reload_ext gams_magic
%gams_cleanup -k
%gams_reset

In [2]:
%%gams
$title turning network

option limcol = 0, limrow = 0, solprint = off;
set
    nodes
    roadID
    season
    seriousness
;

alias (nodes,i,j);
$gdxin ../turning_network_time_updated.gdx
$loadm nodes=dim1 nodes=dim2
parameter real_distance(nodes,nodes) distance of a road;
$load  real_distance=time
$gdxin


$gdxin ../turning_network_turn_updated.gdx
parameter turn(nodes,nodes) distance of a road;
$load  turn=turn
$gdxin


*display distance;


$gdxin ../turning_network_linkID_updated.gdx
set road(roadID<,i,j);
$load  road=road
$gdxin



$ontext
$gdxin ../crash_file.gdx
parameter  crash(nodes,nodes) number of crashes on a road;
$load  crash=crash
$gdxin
$offtext

$ontext
$gdxin ../crash_seriousness.gdx
$load season=dim3 seriousness=dim4
parameter crashS(nodes,nodes,season,seriousness) number of crashes on a road with different seasons and seriousness;
$load  crashS=serious_crash
$gdxin
$offtext

parameter distance(nodes,nodes);

distance(i,j) = real_distance(i,j);


set arc(nodes,nodes);

arc(i,j) = no;
arc(i,j)$(distance(i,j) > 0.5) = yes;
* adjust the distance
distance(i,j)$(arc(i,j)) = distance(i,j) -1;

scalar
    turn_const /100/
;

parameter
    supply(nodes)
;

scalar origin,destination;
execseed = 1 + gmillisec(jnow);
origin = uniformint(1,card(nodes));
destination = uniformint(1,card(nodes));

supply(nodes)$(ord(nodes) = origin) = 1;

supply(nodes)$(ord(nodes) = destination) = -1;

*supply('1666494_2') = 1;
*supply('1662728_3') = -1;

free variable
    total_dist
;

integer variable
    flow(i,j)
;

flow.lo(i,j) = 0

equation
    balance(nodes)
    objective_shortestPath
    objective_safestPath
    objective_shortestPath2
    turn_constr
;


balance(i)..
    sum(arc(i,j), flow(i,j)) - sum(arc(j,i), flow(j,i)) =e= supply(i);
    
objective_shortestPath..
    total_dist =e= sum(arc(i,j),flow(i,j)*turn(i,j));

objective_shortestPath2..
    total_dist =e= sum(arc(i,j),flow(i,j)*distance(i,j) + 0*flow(i,j)*turn(i,j));
    
turn_constr..
    sum(arc(i,j),flow(i,j)*turn(i,j)) =l= turn_const;


model seasonalPath /balance, objective_shortestPath/;

model seasonalPath2 /balance, objective_shortestPath2, turn_constr/;

solve seasonalPath2 using mip minimizing total_dist;


set roadChosen(roadID);

roadChosen(roadID) = no;

scalar roadChosen2_turn;

scalar 
    roadChosen_time
    roadChosen_turn
;

roadChosen_time = sum(arc(i,j),flow.l(i,j)*real_distance(i,j));
roadChosen_turn = sum(arc(i,j),flow.l(i,j)*turn(i,j));

roadChosen(roadID)$( sum(road(roadID,i,j), flow.l(i,j)) > 0.5) = yes;
roadChosen(roadID)$( sum(road(roadID,i,j), flow.l(j,i)) > 0.5) = yes;

*distance(i,j) = turn(i,j);

solve seasonalPath using mip minimizing total_dist;
set roadChosen2(roadID);
scalar roadChosen2_time;

roadChosen2_time = sum(arc(i,j),flow.l(i,j)*real_distance(i,j));
roadChosen2_turn = sum(arc(i,j),flow.l(i,j)*turn(i,j));

roadChosen2(roadID) = no;

roadChosen2(roadID)$( sum(road(roadID,i,j), flow.l(i,j)) > 0.5) = yes;
roadChosen2(roadID)$( sum(road(roadID,i,j), flow.l(j,i)) > 0.5) = yes;


set roadChosen3(roadID);
turn_const = (roadChosen2_turn + roadChosen_turn)/2;

solve seasonalPath2 using mip minimizing total_dist;
roadChosen3(roadID) = no;

roadChosen3(roadID)$( sum(road(roadID,i,j), flow.l(i,j)) > 0.5) = yes;
roadChosen3(roadID)$( sum(road(roadID,i,j), flow.l(j,i)) > 0.5) = yes;

scalar 
    roadChosen3_time
    roadChosen3_turn
;
roadChosen3_time = sum(arc(i,j),flow.l(i,j)*real_distance(i,j));
roadChosen3_turn = sum(arc(i,j),flow.l(i,j)*turn(i,j));

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),566.9937,3212,9043,MIP,CPLEX,1.26
1,Normal (1),Optimal Global (1),5.0000,3211,9043,MIP,CPLEX,0.146
2,Normal (1),Optimal Global (1),573.1778,3212,9043,MIP,CPLEX,0.347


In [3]:
%gams_pull -d flow
%gams_pull supply
%gams_pull -d road
%gams_pull roadChosen
%gams_pull roadChosen2
%gams_pull roadChosen3
%gams_pull roadChosen_time
%gams_pull roadChosen_turn
%gams_pull roadChosen2_time
%gams_pull roadChosen2_turn
%gams_pull roadChosen3_time
%gams_pull roadChosen3_turn

In [48]:
# get the chosen road
chosen_road_short = []
chosen_road_turn = []
chosen_road_intermediate = []
for link in roadChosen:
    if "intersection" not in link:
        chosen_road_short.append(int(link)) 
        
for link in roadChosen2:
    if "intersection" not in link:
        chosen_road_turn.append(int(link))
        
for link in roadChosen3:
    if "intersection" not in link:
        chosen_road_intermediate.append(int(link)) 
# get the chosen origin and destination
origin = None
destination = None
for i in supply:
    if i[1] == 1:
        origin = int(i[0].split("_")[0])
    else:
        destination = int(i[0].split("_")[0]) 

In [49]:
plt.rcParams["figure.figsize"] = (20,10)
plt.figure()

    
chosen_turn = []
chosen_short = []
chosen_intermediate = []
    
#plot original map 
for sp in shape.shapeRecords():
    road_id = sp.record[43]

    x = [i[0] for i in sp.shape.points[:]]
    y = [i[1] for i in sp.shape.points[:]]
        
    #record chosen road
    if road_id in chosen_road_turn:
        segment = []
        for i in range(len(x)):
                segment.append((y[i],x[i]))
        chosen_turn.append(segment)
    
    if road_id in chosen_road_short:
        segment = []
        for i in range(len(x)):
                segment.append((y[i],x[i]))
        chosen_short.append(segment)
        
    if road_id in chosen_road_intermediate:
        segment = []
        for i in range(len(x)):
                segment.append((y[i],x[i]))
        chosen_intermediate.append(segment)

<Figure size 1440x720 with 0 Axes>

In [50]:
m = folium.Map(location=[43.0819, -89.5579])

for loc in chosen_short:
    folium.PolyLine(loc,
                color='red',
                weight=4,
                opacity=0.8).add_to(m)
    
for loc in chosen_turn:
    folium.PolyLine(loc,
                color='blue',
                weight=3,
                opacity=0.8).add_to(m)
    
for loc in chosen_intermediate:
    folium.PolyLine(loc,
                color='green',
                weight=2,
                opacity=0.8).add_to(m)

In [51]:
m

In [44]:
print(roadChosen_turn)
print(roadChosen2_turn)
print(roadChosen3_turn)

print(roadChosen_time)
print(roadChosen2_time)
print(roadChosen3_time)

[11.0]
[9.0]
[9.0]
[1426.8178333982135]
[1644.381140862515]
[1423.8178333982135]


In [19]:
#m.save("turn_vs_time_example2.html")

In [290]:
#supply('1666494_2') = 1;
#supply('1662728_3') = -1;

#1662247 1662247